In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import yaml
import os

In [ ]:
os.chdir('../notebook')
from lane_line import LaneLine, predict_other_lane_line, middle_lane_fit, middle_lane_fit_weighted

In [ ]:
# Move up to the parent directory and then into the 'utils' folder
os.chdir('../utils')

# Verify the new working directory
print("New working directory:", os.getcwd())

In [ ]:
from camera_warp import warp_image_path, warp_image_undistorted
from camera_calib import undistort_image, undistort_image_path

# Load calibration data
with open('camera_calibration.yaml', 'r') as f:
    calibration_data = yaml.safe_load(f)

camera_matrix = np.array(calibration_data['camera_matrix'])
dist_coeffs = np.array(calibration_data['dist_coeff'])

camera_matrix, dist_coeffs

In [ ]:
import os
# Expand tilde (~) to full user home directory
image_path = os.path.expanduser(r"~\OneDrive - McMaster University\AutoRC\2025-03-12\gt_p1_h0_d0.jpg")

test_undistorted, _ = undistort_image_path(image_path)

# Call the function
warped_img, M, Minv = warp_image_path(image_path)

In [ ]:
# original_img = cv2.imread(image_path)
# cv2.imshow('Original Image', original_img)
# cv2.imshow('Undistorted Image', test_undistorted)
# cv2.imshow('Bird\'s Eye View', warped_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
# Color space transformation

def color_space_transform(image):
    # Convert to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define lower and upper thresholds for white
    lower_white = np.array([0, 0, 180])   # Minimum white (high Value)
    upper_white = np.array([180, 50, 255]) # Maximum white (low Saturation)
    
    # Thresholding to extract white
    mask = cv2.inRange(hsv, lower_white, upper_white)
    
    kernel = np.ones((3,3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    edges = cv2.Canny(mask, 50, 150)  # Adjust thresholds as needed
    
    return mask, edges

In [ ]:
# Copy from https://github.com/laavanyebahl/Advance-Lane-Detection-and-Keeping-
# Modified using claude.ai to work with newer NumPy versions

def sliding_window_polyfit(image):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(image[image.shape[0]//2:,:], axis=0)
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = int(histogram.shape[0]//2)
    # Previously the left/right base was the max of the left/right half of the histogram
    # FIXME!! NOT SUITABLE FOR OUR  CONDITION
    leftx_base = np.argmax(histogram[0:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    # Choose the number of sliding windows
    nwindows = 10
    # Set height of windows
    window_height = int(image.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = image.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 30
    # Set minimum number of pixels found to recenter window
    minpix = 60
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    # Rectangle data for visualization
    rectangle_data = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = image.shape[0] - (window+1)*window_height
        win_y_high = image.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        rectangle_data.append((win_y_low, win_y_high, win_xleft_low, win_xleft_high, win_xright_low, win_xright_high))
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    left_fit, right_fit = (None, None)
    # Fit a second order polynomial to each
    if len(leftx) != 0:
        left_fit = np.polyfit(lefty, leftx, 2)
    if len(rightx) != 0:
        right_fit = np.polyfit(righty, rightx, 2)
    
    display_data = (rectangle_data, histogram)
    
    return left_fit, right_fit, left_lane_inds, right_lane_inds, display_data

In [ ]:
# Build our own optimized polyfit function instead
# def polyfit_using_prev_fit(binary_warped, left_fit, right_fit):
#     nonzero = binary_warped.nonzero()
#     nonzeroy = np.array(nonzero[0])
#     nonzerox = np.array(nonzero[1])
#     margin = 60
#     left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
#     left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
#     left_fit[1]*nonzeroy + left_fit[2] + margin))) 

#     right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
#     right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
#     right_fit[1]*nonzeroy + right_fit[2] + margin)))  

#     # Again, extract left and right line pixel positions
#     leftx = nonzerox[left_lane_inds]
#     lefty = nonzeroy[left_lane_inds] 
#     rightx = nonzerox[right_lane_inds]
#     righty = nonzeroy[right_lane_inds]
#     # Fit a second order polynomial to each
#     left_fit_new, right_fit_new = (None, None)
#     if len(leftx) != 0:
#         # Fit a second order polynomial to each
#         left_fit_new = np.polyfit(lefty, leftx, 2)
#     if len(rightx) != 0:
#         right_fit_new = np.polyfit(righty, rightx, 2)
    
#     return left_fit_new, right_fit_new, left_lane_inds, right_lane_inds

In [ ]:
masked, images = color_space_transform(warped_img)

In [ ]:
left_fit, right_fit, left_lane_inds, right_lane_inds, display_data = sliding_window_polyfit(masked)

In [ ]:
def plot_polynomial_on_image(img, coeffs, color='red'):
    height, width = img.shape[:2]
    x = np.linspace(0, width-1, width)
    y = np.polyval(coeffs, x)
    

In [ ]:
left_fit, right_fit

In [ ]:
# Set up plot
fig, axs = plt.subplots(1, 3, figsize=(20, 4))
axs = axs.ravel()

h = test_undistorted.shape[0]

left_fit_x_int = left_fit[0]*h**2 + left_fit[1]*h + left_fit[2]
right_fit_x_int = right_fit[0]*h**2 + right_fit[1]*h + right_fit[2]

rectangles = display_data[0]
histogram = display_data[1]

# Create an output image to draw on and  visualize the result
out_img = np.uint8(np.dstack((masked, masked, masked))*255)
# Generate x and y values for plotting
ploty = np.linspace(0, masked.shape[0]-1, masked.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
for rect in rectangles:
# Draw the windows on the visualization image
    cv2.rectangle(out_img,(rect[2],rect[0]),(rect[3],rect[1]),(0,255,0), 2) 
    cv2.rectangle(out_img,(rect[4],rect[0]),(rect[5],rect[1]),(0,255,0), 2) 
# Identify the x and y positions of all nonzero pixels in the image

nonzero = masked.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])
out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [100, 200, 255]

axs[0].imshow(warped_img, cmap='gray')
axs[1].imshow(out_img, cmap='gray')
axs[1].plot(left_fitx, ploty, color='yellow')
axs[1].plot(right_fitx, ploty, color='yellow')

axs[2].plot(histogram)

In [ ]:
''' NOTE :
 xm_per_pixel = 0.36/117 ( 0.36 actual width of lane, 117 px on warped image)
 ym_per_pixel = 0.2 / 37 (0.2 m actual length of black line, 37 px length on warped image) 
'''
def calculate_rad_curvature_and_position(binary_warped, l_fit, r_fit, l_lane_inds, r_lane_inds):
    # Define conversions in x and y from pixels space to meters
    xm_per_pix = 0.36 /117
    ym_per_pix = 0.2 / 37
    left_curverad, right_curverad, center_dist = (0, 0, 0)
    # Define y-value where we want radius of curvature
    # I'll choose the maximum y-value, corresponding to the bottom of the image
    h = binary_warped.shape[0]
    ploty = np.linspace(0, h-1, h)
    y_eval = np.max(ploty)
  
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Again, extract left and right line pixel positions
    leftx = nonzerox[l_lane_inds]
    lefty = nonzeroy[l_lane_inds] 
    rightx = nonzerox[r_lane_inds]
    righty = nonzeroy[r_lane_inds]
    
    if len(leftx) != 0 and len(rightx) != 0:
        # Fit new polynomials to x,y in world space
        left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
        right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
        # Calculate the new radii of curvature
        left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
        right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
        # Now our radius of curvature is in meters
        
    # Distance from center is image x midpoint - mean of l_fit and r_fit intercepts 
    if r_fit is not None and l_fit is not None:
        car_position = binary_warped.shape[1]/2
        l_fit_x_int = l_fit[0]*h**2 + l_fit[1]*h + l_fit[2]
        r_fit_x_int = r_fit[0]*h**2 + r_fit[1]*h + r_fit[2]
        lane_center_position = (r_fit_x_int + l_fit_x_int) /2
        center_dist = (car_position - lane_center_position) * xm_per_pix
    
    return left_curverad, right_curverad, center_dist

In [ ]:
rad_curv_left, rad_curv_right, offset = calculate_rad_curvature_and_position(warped_img, left_fit, right_fit, left_lane_inds, right_lane_inds)

In [ ]:
rad_curv_left, rad_curv_right, offset

In [ ]:
def select_points_in_windows_vectorized(masked_image, search_windows, weighted=False):
    # Get all white points (nonzero pixels) from the binary image
    white_points = np.column_stack(np.nonzero(masked_image))
    
    # Initialize an empty array to collect all points in windows
    all_points = []
    
    # Process each window
    for window in search_windows:
        if weighted:
            top_left, bottom_right, _ = window
        else:
            top_left, bottom_right = window
            
        min_col, min_row = top_left
        max_col, max_row = bottom_right
        
        # Create mask for points within this window
        mask = ((white_points[:, 0] >= min_row) & 
                (white_points[:, 0] <= max_row) & 
                (white_points[:, 1] >= min_col) & 
                (white_points[:, 1] <= max_col))
        
        # Add points from this window
        points_in_this_window = white_points[mask]
        if len(points_in_this_window) > 0:
            all_points.append(points_in_this_window)
    
    # Combine all points and remove duplicates if needed
    if all_points:
        all_points = np.unique(np.vstack(all_points), axis=0)
        return all_points
    else:
        return np.array([])
        
def select_points_and_fit_curve(masked_image, search_windows, weighted=False, degree=2):
    all_points = select_points_in_windows_vectorized(masked_image, search_windows, weighted)

    if len(all_points) > 0:
        points_in_windows = np.unique(np.vstack(all_points), axis=0)
        
        # Extract y and x coordinates for the polynomial fit
        # Note: y is typically the row (first coordinate) and x is the column (second coordinate)
        lefty = points_in_windows[:, 0]  # rows
        leftx = points_in_windows[:, 1]  # columns
        
        # Fit polynomial
        if len(lefty) >= degree + 1:  # Need at least degree+1 points to fit a polynomial of degree
            coefficients = np.polyfit(lefty, leftx, degree)
            return points_in_windows, coefficients
        else:
            return points_in_windows, None
    else:
        return np.array([]), None

In [ ]:
import numpy as np

def check_overlap(window1, window2):
    """
    Check if two windows overlap.
    Each window is represented by its top-left (x1, y1) and bottom-right (x2, y2) coordinates.
    
    Args:
        window1: tuple of ((top_left_x, top_left_y), (bottom_right_x, bottom_right_y))
        window2: tuple of ((top_left_x, top_left_y), (bottom_right_x, bottom_right_y))
    
    Returns:
        True if windows overlap, False otherwise
    """
    # Extract coordinates
    (x1_tl, y1_tl), (x1_br, y1_br) = window1
    (x2_tl, y2_tl), (x2_br, y2_br) = window2
    
    # Check if one window is to the left of the other
    if x1_br < x2_tl or x2_br < x1_tl:
        return False
    
    # Check if one window is above the other
    if y1_br < y2_tl or y2_br < y1_tl:
        return False
    
    # If neither of the above conditions is true, windows overlap
    return True

def remove_overlapping_windows(windows_a, windows_b, mode='remove_a_keep_b'):
    """
    Remove windows that overlap between two sets based on the specified mode.
    
    Args:
        windows_a: numpy array of windows, each window is ((top_left_x, top_left_y), (bottom_right_x, bottom_right_y))
        windows_b: numpy array of windows, each window is ((top_left_x, top_left_y), (bottom_right_x, bottom_right_y))
        mode: string that controls which windows to remove:
              'remove_a_keep_b' - remove overlapping windows from set A, keep all windows in set B
              'remove_b_keep_a' - remove overlapping windows from set B, keep all windows in set A
    
    Returns:
        If mode is 'remove_a_keep_b': filtered_windows_a, windows_b
        If mode is 'remove_b_keep_a': windows_a, filtered_windows_b
    """
    if mode not in ['remove_a_keep_b', 'remove_b_keep_a']:
        raise ValueError("Mode must be either 'remove_a_keep_b' or 'remove_b_keep_a'")
    
    if mode == 'remove_a_keep_b':
        # Remove overlapping windows from set A
        non_overlapping_a = []
        
        for window_a in windows_a:
            overlaps = False
            
            for window_b in windows_b:
                if check_overlap(window_a, window_b):
                    overlaps = True
                    break
            
            if not overlaps:
                non_overlapping_a.append(window_a)
        
        # Convert list back to numpy array
        if non_overlapping_a:
            filtered_a = np.array(non_overlapping_a)
        else:
            filtered_a = np.empty((0, 2, 2), dtype=windows_a.dtype)
            
        return filtered_a, windows_b
    
    else:  # mode == 'remove_b_keep_a'
        # Remove overlapping windows from set B
        non_overlapping_b = []
        
        for window_b in windows_b:
            overlaps = False
            
            for window_a in windows_a:
                if check_overlap(window_b, window_a):
                    overlaps = True
                    break
            
            if not overlaps:
                non_overlapping_b.append(window_b)
        
        # Convert list back to numpy array
        if non_overlapping_b:
            filtered_b = np.array(non_overlapping_b)
        else:
            filtered_b = np.empty((0, 2, 2), dtype=windows_b.dtype)
            
        return windows_a, filtered_b

In [ ]:
# Open the video file
# OpenCV's cv2.VideoCapture() often struggles to open H.264-encoded videos directly, 
# especially when they are recorded using Picamera2. This happens because OpenCV does 
# not natively support H.264 due to licensing restrictions. Here’s how you can fix it:
# ffmpeg -i <input.h264> -c:v copy <output.mp4>
# or can use processed video: ~\OneDrive - McMaster University\AutoRC\2025-03-12\processed

# Expand tilde (~) to full user home directory
video_path = os.path.expanduser(r"~\OneDrive - McMaster University\AutoRC\2025-03-19\processed\loop2.mp4")

# Check if the file exists
if os.path.exists(video_path):
    print("File exists:", video_path)
else:
    print("File does not exist:", video_path)
    
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

initialized = False
previous_left_line = None
previous_right_line = None

# Process video frame by frame
while True:
    # Read a frame from the video
    ret, frame = cap.read()
    
    # If no frame is read (end of video), break
    if not ret:
        print("End of video or error reading frame.")
        break

    img, _ = undistort_image(frame, camera_matrix, dist_coeffs)
    warped, _, _ = warp_image_undistorted(img)

    masked, edges = color_space_transform(warped)

    nonzero = masked.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    out_img = np.uint8(np.dstack((masked, masked, masked))*255)
    
    if initialized:
        # For the polyfit we can actually assign larger weight to
        # near (a.k.a bottom region) points and assign lighter weight to far points in order to eliminate noise
        # Currently not implemented so keep it False
        weighted=False
        
        left_search_windows = previous_left_line.sliding_window_loc(masked, margin=30, weighted=weighted)
        right_search_windows = previous_right_line.sliding_window_loc(masked, margin=30, weighted=weighted)

        # if len(left_search_windows) < len(right_search_windows) and previous_right_line.is_turning_left(masked.shape[0]):
        if len(left_search_windows) != len(right_search_windows) and previous_right_line.is_turning_left(masked.shape[0]):
            left_search_windows, right_search_windows = remove_overlapping_windows(left_search_windows, right_search_windows, mode='remove_a_keep_b')
        elif len(left_search_windows) != len(right_search_windows) and previous_left_line.is_turning_right(masked.shape[0]):
            left_search_windows, right_search_windows = remove_overlapping_windows(left_search_windows, right_search_windows, mode='remove_b_keep_a')

        # Draw the windows after overlap removal, for better visualization and debug
        for window in left_search_windows:
            (top_left_x, top_left_y), (bottom_right_x, bottom_right_y) = window
            cv2.rectangle(out_img,(top_left_x,top_left_y),(bottom_right_x,bottom_right_y),(0,255,0), 2) 

        for window in right_search_windows:
            (top_left_x, top_left_y), (bottom_right_x, bottom_right_y) = window
            cv2.rectangle(out_img,(top_left_x,top_left_y),(bottom_right_x,bottom_right_y),(255,0,0), 2)

        # Try with polyfit first
        points_in_left, left_fit_cos = select_points_and_fit_curve(masked, left_search_windows, weighted)
        points_in_rigth, right_fit_cos = select_points_and_fit_curve(masked, right_search_windows, weighted)

        if left_fit_cos is not None and right_fit_cos is not None:
            # All two line are visiable
            #print('Condition: 1')
            left_fit = LaneLine('left', 'measured', (left_fit_cos[0], left_fit_cos[1], left_fit_cos[2]))
            right_fit = LaneLine('right', 'measured', (right_fit_cos[0], right_fit_cos[1], right_fit_cos[2]))
            # if (3 <= len(left_search_windows) <= 7):
            #     left_fit = middle_lane_fit(left_fit, predict_other_lane_line(right_fit, masked))
            #     print('Using predict as smoothing: Left', left_fit)
            # elif (3 <= len(right_search_windows) <= 7):
            #     right_fit = middle_lane_fit(right_fit, predict_other_lane_line(left_fit, masked))
            #     print('Using predict as smoothing: Right', right_fit)
            if (len(left_search_windows) <= 7):
                left_fit = predict_other_lane_line(right_fit, masked)
                print('Using predict only: Left', left_fit);
            elif (len(right_search_windows) <= 7):
                right_fit = predict_other_lane_line(left_fit, masked)
                print('Using predict only: Right', right_fit);
        elif right_fit_cos is not None and left_fit_cos is None:
            # Only right line is visible
            print('Condition: 2')
            right_fit = LaneLine('right', 'measured', (right_fit_cos[0], right_fit_cos[1], right_fit_cos[2]))
            left_fit = predict_other_lane_line(right_fit, masked)
        elif left_fit_cos is not None and right_fit_cos is None:
            # Only left line is visible
            print('Condition: 3')
            left_fit = LaneLine('left', 'measured', (left_fit_cos[0], left_fit_cos[1], left_fit_cos[2]))
            right_fit = predict_other_lane_line(left_fit, masked)
        else:
            # Two lines are both invisible
            print('Condition: 4')
            # raise Exception("Sorry, no found!")
            initialized = False
            break
            
    else:
        # First time, initialized with sliding_window_method
        left_fit_cos, right_fit_cos, left_lane_inds, right_lane_inds, _ = sliding_window_polyfit(masked)
        left_fit = LaneLine('left', 'measured', (left_fit_cos[0], left_fit_cos[1], left_fit_cos[2]))
        right_fit = LaneLine('right', 'measured', (right_fit_cos[0], right_fit_cos[1], right_fit_cos[2]))
        initialized = True
        print('Initialized!')

    previous_left_line = left_fit
    previous_right_line = right_fit
    middle_line = middle_lane_fit(left_fit, right_fit)

    # Draw polyfit lines for better visualization and debugging
    ploty = np.linspace(0, masked.shape[0]-1, masked.shape[0])
    for idx, line in enumerate([left_fit, middle_line, right_fit]):
        plotx = line.evaluate(ploty)
        # Convert floating point values to integers for indexing
        plotx = np.round(plotx).astype(np.int64)
        
        # Filter out any points that might be outside the image bounds
        valid_points = (plotx >= 0) & (plotx < masked.shape[1])
        ploty_valid = ploty[valid_points].astype(np.int64)
        plotx_valid = plotx[valid_points]
        
        # Draw the line
        out_img[ploty_valid, plotx_valid] = [100, 200, 255] if idx == 1 else [255, 100, 200]

    cv2.imshow('Video Frame', frame)
    cv2.imshow('Bird\'s Eye View', warped)
    cv2.imshow('Masked Image', out_img)
    
    # Wait for 25ms and check for 'q' key to exit
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()